In [2]:
from tkinter import *
from random import *
from tkinter import messagebox
import re
import pyperclip
import json

In [3]:
#-----CONSTANTS---------------------------
CHAR_SHIFT = randint(1, 27)

In [26]:
#---------Pasword generator---------------
def generate_password(): 
    letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
    numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
    symbols = ['`','!', '@', '#', '$', '%', '^' '&', '(', ')', '*', '+', '-']

    password_list = []
    
    password_letter = [choice(letters) for i in range(randint(8, 10))]
    password_sym = [choice(symbols) for i in range(randint(2, 4))]
    password_numbers = [choice(numbers) for i in range(randint(2, 4))]

    password_list = password_letter + password_sym + password_numbers
    shuffle(password_list)

    password = "".join(password_list)
    password_e.insert(0, password)
    
    pyperclip.copy(password)

    print(f"Your password is: {password}")

In [27]:
#---VALIDATION METHODS--------------------
#___password Cipher_______________________
def __encrypt(text, shift):
    result = ""       #cipher text
    # transverse the plain text
    for i in range(len(text)):
        char = text[i]
        if not char.isalpha():
            result += char
        # Encrypt uppercase characters in plain text
        elif (char.isupper()):
            result += chr((ord(char) + shift - 65) % 26 + 65)   #chg to ord(char)-shift-65) to dicypher
        # Encrypt lowercase characters in plain text
        else:
            result += chr((ord(char) + shift - 97) % 26 + 97)   #chg to ord(char)-shift-65) to dicypher
    return result

#---Dicipher------------------------------
def __decrypt(text, shift):
    result = ""
    for i in range(len(text)):
        char = text[i]
        if not char.isalpha():
            result += char
        elif (char.isupper()):
            result += chr((ord(char) - shift - 65) % 26 + 65)
        else:
            result += chr((ord(char) - shift - 97) % 26 + 97)
    return result

#---Validate email------------------------
def __check_email(email):
    if len(email) < 6:
        return False
    email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    if (re.fullmatch(email_regex, email)):
        return True
    return False

#---Validate password---------------------

#Conditions for a valid password are:
# 1. Should have at least one number.
# 2. Should have at least one uppercase and one lowercase character.
# 3. Should have at least one special symbol.
# 4. Should be between 6 to 20 characters long.

def __check_password(password):
    pass_regex = r"^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*#?&])[A-Za-z\d@$!#%*?&]{6,}$"
    if (re.fullmatch(pass_regex, password)):
        return True
    return False

In [33]:
#---------Save password-------------------
def save():
    #get string from entry box
    website = website_e.get()
    email = username_e.get()
    password = password_e.get()
    
    #validate email address
    is_email_valid = __check_email(email)
    is_pass_valid = __check_password(password)
    
    if not is_email_valid or not is_pass_valid:
        messagebox.showerror(title="Email or password Invalid", message="You entered an invalid email and/or password")
        #pop-up email not valid
    else:
        #msg box
        is_ok = messagebox.askokcancel(title=website, message=f"Verify the information provided:\nWebsite: {website}\nEmail: {email}\nPassword: {password}\nDo you wan to save?\n")
        
        if is_ok:
            #encrypt password
            password = __encrypt(password, CHAR_SHIFT)
            dict_d = {
                website: {
                    "email": email,
                    "password": password,
                    "shift": CHAR_SHIFT
                        }
                    }
            
            #write to file
            try:
                with open("data.json", "r") as pass_manager_file:
                    #read old data
                    data = json.load(pass_manager_file)   #<--read data
            except FileNotFoundError:
                with open("data.json","w") as pass_manager_file:
                    json.dump(dict_d, pass_manager_file, indent=2)
            
            else:
                #update old data
                data.update(dict_d)
                with open("data.json", "w") as pass_manager_file:
                    #writing new data
                    json.dump(data, pass_manager_file, indent=2)   #<--write data
            
            finally:
                website_e.delete(0,END)
                password_e.delete(0,END)
                #pass_manager_file.write(f"{website}\t{email}\t{password}\t{CHAR_SHIFT}\n")
                website_e.delete(0,END)
                password_e.delete(0,END)

In [34]:
#----------Find password--------------------
def find_password():
    website = website_e.get()
    try:
        with open("data.json","r") as data_file:
            data = json.load(data_file)
    except FileNotFoundError:
        messagebox.showinfo(title="Error", message="File not found: No such file or directory")
    else:
        if website in data:
            email = data[website]["email"]
            passw = data[website]["password"]
            shift = data[website]["shift"]
            passw = __decrypt(passw, shift)

            messagebox.showinfo(title=website, message=f"Email: {email}\nPassword: {passw}")
        else:
            messagebox.showinfo(title="Error", message=f"Nor details for {website} exists")
            

In [35]:
#-----------CLOSE WINDOW----------------------
def close():
    window.destroy()

In [43]:
#---------UI setup------------------------
#--------WINDOW---------------------------
window = Tk()
window.title("Password Manager")
window.config(padx=50, pady=50, bg="#F6FBF4")

#--------Canvas---------------------------
canvas = Canvas(width=300, height=300, bg="#F6FBF4", highlightthickness=0)
img = PhotoImage(file="logo.png")
canvas.create_image(150, 150, image=img)
canvas.grid(column=1, row=0)

##-------LABELS---------------------------
website_l = Label(text="Website", font=("Times New Roman", 15))
website_l.grid(column=0,row=1)

username_l = Label(text="Email/username:", font=("Times New Roman", 15))
username_l.grid(column=0,row=2)

password_l = Label(text="Password:", font=("Times New Roman", 15))
password_l.grid(column=0, row=3)

#--------Entry-----------------------------
website_e = Entry(window, bd=1, width=35)
website_e.grid(column=1, row=1, columnspan=3)
website_e.focus()

username_e = Entry(window, bd=1, width=35)
username_e.grid(column=1, row=2, columnspan=3)
username_e.insert(0, "yourName@yourThing.your")

password_e = Entry(window, bd=1, width=21)
password_e.grid(column=1, row=3)

#--------Button-----------------------------
add_btn = Button(text="Add", width=36, command=save)
add_btn.config(padx=5, pady=5)
add_btn.grid(column=1, row=4, columnspan=3)

pass_gen_btn = Button(text="Generate Password", command=generate_password)
pass_gen_btn.config(padx=5, pady=5)
pass_gen_btn.grid(column=2, row=3)

search_btn = Button(text="Search", width=10, command=find_password)
search_btn.config(padx=5, pady=5)
search_btn.grid(column=2, row=1)

#-TODO - get all websites usernames password 
#get_all_btn = Button(text="Get All website, username n password", command=get_all_data)
#get_all_btn.config(padx=5, pady=5)
#get_all_btn.grid(column=1, row=5, columnspan=3)


close_btn = Button(text="Close", font=("Times New Roman", 12, "bold"),width=28, command=close)
close_btn.config(padx=5, pady=5)
close_btn.grid(column=1, row=6, columnspan=3)


window.mainloop()